This file allows to analyze results obtained by running `experiments_nsc.py`.

It computed performance metric, analyse the evolution of likelihood given number of clusters if available, and display the obtained clusters (for the selected methodology).

In [ ]:
import os 
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib.colors as mcolors

import sys
sys.path.append('../')
sys.path.append('../NSC/')
sys.path.append('../DeepSurvivalMachines/')

from ntc import datasets
from NSC.experiment import Experiment

In [ ]:
# Change this to analyze other datasets result
dataset = 'METABRIC'

In [ ]:
path = '../Results/' # Path where the data is saved
x, t, e, covariates = datasets.load_dataset(dataset) # Open the data

In [ ]:
horizons = [0.25, 0.5, 0.75] # Horizons to evaluate the models
times_eval = np.quantile(t[e > 0], horizons)

In [ ]:
from pycox.evaluation import EvalSurv
from sksurv.metrics import concordance_index_ipcw, brier_score, cumulative_dynamic_auc, integrated_brier_score

### Utils: The evaluatino metrics used
def evaluate(survival, e = e, t = t,  times_eval = []):
    folds = survival[('Use',)].values.flatten()
    survival = survival.drop(columns = ['Use', 'Assignment'])
    survival.columns = pd.MultiIndex.from_frame(pd.DataFrame(index=survival.columns).reset_index().astype(float))
    times = survival.columns.get_level_values(1).unique()
    results = {}

    # If multiple risk, compute cause specific metrics
    for r in survival.columns.get_level_values(0).unique():
        for fold in np.arange(np.unique(folds).shape[0]):
            res = {}
            e_train, t_train = e[folds != fold], t[folds != fold]
            e_test,  t_test  = e[folds == fold], t[folds == fold]

            et_train = np.array([(e_train[i] == int(r), t_train[i]) for i in range(len(e_train))], # For estimation censoring
                            dtype = [('e', bool), ('t', float)])
            et_test = np.array([(e_test[i] == int(r), t_test[i]) for i in range(len(e_test))], # For measure performance for given outcome
                            dtype = [('e', bool), ('t', float)])
            
            selection = (t_test < t_train.max()) | (e[folds == fold] != int(r))
            
            et_test = et_test[selection]
            survival_train = survival[folds != fold][r]
            survival_fold = survival[folds == fold][r]

            km = EvalSurv(survival_train.T, t_train, e_train == int(r), censor_surv = 'km')
            test_eval = EvalSurv(survival_fold.T, t_test, e_test == int(r), censor_surv = km)

            res['Overall'] = {
                    "CIS": test_eval.concordance_td(), 
                }
            try:
                res['Overall']['BRS'] = test_eval.integrated_brier_score(times.to_numpy())
            except: pass

            
            if len(times_eval) > 0:
                indexes = [np.argmin(np.abs(times - te)) for te in times_eval]
                briers = brier_score(et_train, et_test, survival_fold[selection].iloc[:, indexes], times_eval)[1]
                for te, brier, index in zip(times_eval, briers, indexes):
                    try:
                        res[te] = {
                            "CIS": concordance_index_ipcw(et_train, et_test, 1 - survival_fold[selection].iloc[:, index], te)[0], 
                            "BRS": brier,
                            "ROCS": cumulative_dynamic_auc(et_train, et_test, 1 - survival_fold[selection].iloc[:, index], te)[0][0]}
                    except:
                        pass
                
            results[(r, fold)] = pd.DataFrame.from_dict(res)
    results = pd.concat(results)
    results.index.set_names(['Risk', 'Fold', 'Metric'], inplace = True)

    return results

In [ ]:
# Open file and compute performance
predictions, clusters, results, likelihood = {}, {}, {}, {}
for file_name in os.listdir(path):
    if dataset in file_name and '.csv' in file_name: 
        model = file_name       
        model = model[model.index('_') + 1: model.index('.')]

        print("Opening :", file_name, ' - ', model)
        predictions[model] = pd.read_csv(path + file_name, header = [0, 1], index_col = 0)
        results[model] = evaluate(predictions[model], times_eval = times_eval)

# Rename
# TODO: Add your method in the list for nicer display
dict_name = {'nsc': 'NSC', 'cox': 'CoxPH', 'ds': 'DeepSurv', 'dsm': 'DSM', 'dcm': 'DCM', 'dh': 'DeepHit', 'sumo': 'SuMo', 'st': 'Suvival Tree'} 

likelihood = pd.DataFrame.from_dict(likelihood, 'index').rename(dict_name)
results = pd.concat(results).rename(dict_name)
results.index.set_names('Model', 0, inplace = True)

In [ ]:
table = results.groupby(['Model', 'Risk', 'Metric']).apply(lambda x: pd.Series(["{:.3f} ({:.2f})".format(mean, std) for mean, std in zip(x.mean(), x.std())], index = x.columns))
table = table.loc[table.index.get_level_values(2).isin(['CIS', 'BRS'])].unstack(level=-1).stack(level=0).unstack(level=-1).loc[:, ['CIS', 'BRS']]
#table = table.loc[['NSC', 'DCM', 'SuMo', 'DSM', 'DeepHit', 'DeepSurv', 'CoxPH']]

if len(table.index.get_level_values(1).unique()) == 1:
    table = table.droplevel(1)
else:
    table = table.reorder_levels(['Risk', 'Model']).sort_index(level = 0, sort_remaining = False)

table

In [ ]:
print(table.to_latex())

------------

# Likelihood evolution

In [ ]:
# Anlayze the outcome of the clustering method 
method_display = 'nsc' 

In [ ]:
# Load models in family
likelihood = {}
for file_name in os.listdir(path):
    if dataset in file_name and '.pickle' in file_name and method_display in file_name and 'k=' in file_name:
        model = int(file_name[file_name.rindex('k=')+2: file_name.index('.')])
        print("Likelihood Computation :", file_name, ' - ', model)

        model_pickle = Experiment.load(path + file_name)
        likelihood[model] = model_pickle.likelihood(x, t, e)
likelihood = pd.DataFrame.from_dict(likelihood, 'index')

In [ ]:
mean = likelihood.sort_index().mean(1)
std = 1.96 * likelihood.sort_index().std(1) / np.sqrt(5)

mean.plot()
plt.fill_between(std.index, mean + std, mean - std, alpha = 0.3)
plt.grid(alpha = .3)

plt.xlabel(r'Number of clusters $K$')
plt.ylabel(r'Negative Log Likelihood')

---------

# Analysis cluster

In [ ]:
# Anlayze the outcome of the clustering method 
method_display = 'nsc_k=3' 

In [ ]:
# Compute the average cluster (can be for any method)
assignment = {}
ax = None
for i in np.arange(5):
    horizons_pred = np.linspace(0, 0.75, 10)
    pred = predictions[method_display]
    pred = pred[(pred.Use == i).values]

    assignment[i] = pred.Assignment.idxmax(1)
    pred = pred['1']
    pred.columns = pred.columns.map(float)
    ax = pred.groupby(assignment[i]).mean(0).T.plot(ax = ax, ls = '-')
    
plt.xlabel('Time')
plt.ylabel('Survival Predictions')
plt.title('Average Cluster Across Folds')
plt.grid(alpha = 0.3)
plt.show()

In [ ]:
# Load the experiment associated - Only works when predcit_cluster is available
for file_name in os.listdir(path):
    if dataset in file_name and method_display + '.pickle' in file_name:
        print("Cluster Computation :", file_name)

        model_pickle = Experiment.load(path + file_name)
        clusters = model_pickle.survival_cluster(x)

In [ ]:
ax = None
for i in clusters:
    ax = pd.DataFrame(clusters[i], index = model_pickle.times).plot(ax = ax)
plt.xlabel('Time')
plt.ylabel('Survival Predictions')
plt.title('Estimated Cluster')
plt.grid(alpha = 0.3)
plt.show()

In [ ]:
# Compute average clusters across dataset
average, ordering = {}, {}
for fold in clusters:
    horizons_pred = np.linspace(0, 0.75, 10)
    average[fold] = pd.DataFrame(clusters[fold], index = model_pickle.times).rename_axis('Cluster')
    ordering[fold] = {i: j for j, i in enumerate(average[fold].iloc[-1].sort_values().index)}
    average[fold] = average[fold].rename(index = ordering[fold])
else:
    try: 
        average = pd.concat(average, names = ['Fold'])
        mean = average.groupby('Cluster').mean()
        confidence = 1.96 * average.groupby('Cluster').std() / len(average.index.get_level_values('Fold').unique())
        ax = mean.plot()
        for c, color in zip(mean.columns, list(mcolors.TABLEAU_COLORS.values())[:len(mean.columns)]):
            ax.fill_between(mean.index, (mean[c] - confidence[c]), (mean[c] + confidence[c]), color = color, alpha = .1)
        plt.xlabel('Time')
        plt.ylabel('Survival Probability')
        plt.grid(alpha = 0.3)
        plt.legend(title = 'Clusters')
        plt.show()
    except:
        print('Not same number of clusters')

In [ ]:
# Compute the Kaplan Meier of patients in these clusters
from lifelines import KaplanMeierFitter
km_estimate = {}
for fold in clusters:
    selection = (predictions[method_display].Use == fold).values.flatten()
    assignment = predictions[method_display][selection].Assignment.idxmax(1)
    km_estimate[fold] = {}
    for i in assignment.unique():
        km = KaplanMeierFitter().fit(t[selection][assignment == i], e[selection][assignment == i])
        km_estimate[fold][i] = km.survival_function_at_times(model_pickle.times)
    km_estimate[fold] = pd.concat(km_estimate[fold], axis = 1).rename_axis('Cluster').rename(index = ordering[fold])

ax = mean.plot(ls = '--')
km_estimate = pd.concat(km_estimate, names = ['Fold'])
mean = km_estimate.groupby('Cluster').mean()
confidence = 1.96 * km_estimate.groupby('Cluster').std() / len(km_estimate.index.get_level_values('Fold').unique())
mean.plot(ax = ax, color = {i: list(mcolors.TABLEAU_COLORS.values())[int(i)] for i in mean.columns})
for c, color in zip(mean.columns, list(mcolors.TABLEAU_COLORS.values())[:len(mean.columns)]):
    ax.fill_between(mean.index, (mean[c] - confidence[c]), (mean[c] + confidence[c]), color = color, alpha = .1)
plt.xlabel('Time')
plt.ylabel('Survival Estimate')
plt.grid(alpha = 0.3)

handle = [Line2D([0], [0],ls = '--'), Line2D([0], [0], ls = '-')]
plt.legend(handle, ['Kaplan-Meier', 'Estimated Cluster'])
plt.show()

------------

# Cluster Exploration

In [ ]:
# What is the distribution of probability to be part of a given cluster ?
clusters_assignment = {}
for fold in clusters:
    selection = (predictions[method_display].Use == fold).values.flatten()
    clusters_assignment[fold] = predictions[method_display][selection].Assignment.rename(index = ordering[fold])

clusters_assignment = pd.concat(clusters_assignment, axis = 0)
for cluster in clusters_assignment.columns:
    clusters_assignment[cluster].plot.hist(alpha = 0.5, bins = 100)
plt.xlabel('Probality cluster')
plt.grid(alpha = 0.3)
plt.legend(title = 'Clusters')
plt.show()

# Distribution maximally assigned
axes = clusters_assignment.groupby(clusters_assignment.apply(lambda x: np.argmax(x), axis = 1)).boxplot(layout = (1, 3), figsize = (7, 3), grid = 0.5)
for ax in axes:
    ax.grid(alpha = 0.3)

In [ ]:
# Compute average life expectancy for each cluster
from lifelines.statistics import multivariate_logrank_test
clusters_expectancy = []
for fold in clusters:
    cluster_fold = clusters_assignment.loc[fold].copy()
    cluster_fold['Assignment'] = cluster_fold.T.idxmax().T
    cluster_fold['Time'] = t[cluster_fold.index]
    cluster_fold['Event'] = e[cluster_fold.index]
    clusters_expectancy.append(cluster_fold.groupby('Assignment').apply(lambda x: KaplanMeierFitter().fit(x['Time'], x['Event']).median_survival_time_))
    print(multivariate_logrank_test(cluster_fold['Time'], cluster_fold['Assignment'], cluster_fold['Event']))
    print(cluster_fold.groupby('Assignment').mean())
    print(cluster_fold.groupby('Assignment').count())
clusters_expectancy = pd.concat(clusters_expectancy, 1).replace([np.inf, -np.inf], np.nan)
clusters_expectancy

-------

# Cluster Assignment Importance

In [ ]:
# Only available for NSC
for file_name in os.listdir(path):
    if dataset in file_name and method_display + '.pickle' in file_name:
        print("Importance Computation :", file_name)

        model_pickle = Experiment.load(path + file_name)
        importance = model_pickle.importance(x, t, e, n = 1)

In [ ]:
for fold in importance:
    importance[fold] = pd.Series(importance[fold][0])
importance = pd.concat(importance, 1, names = ['Fold'])

In [ ]:
# Display importance of features obtained by test
importance.index = covariates
importance.mean(1).sort_values().plot.bar(yerr = importance.std(1))
plt.xlabel('Covariate')
plt.ylabel('Likelihood change')
plt.grid(alpha = 0.3)